# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    
    n = len(df[label])
    p = 0
    
    for i in list(df[label].value_counts()):
        p += (i/n)**2
    
    
    return 1-p

In [9]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[]
    
    for i in range(1,len(uniques)) :
        result += [list(x) for x in list(combinations(uniques,i))]
    
    return result

In [6]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [52]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    
    n = len(df)
    
    category = get_binary_split(df,attribute)
    
    for i in range(len(category)//2):
        sett1 = df[df[attribute] == category[i][0]]
        sett2 = df[df[attribute] != category[i][0]]
        sutt1 = get_gini(sett1,label)
        sutt2 = get_gini(sett2,label)
        
        result[category[i][0]] = round((len(sett1)*sutt1 + len(sett2)*sutt2)/n,3)
    
    
    return result

In [53]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.443, 'medium': 0.458, 'low': 0.45}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [54]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.443)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [55]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print(f"Minimum Gini index of {feature} : {min(get_attribute_gini_index(pd_data,feature,label).items())}")


Minimum Gini index of age : ('middle_aged', 0.357)
Minimum Gini index of income : ('high', 0.443)
Minimum Gini index of student : ('no', 0.367)
Minimum Gini index of credit_rating : ('fair', 0.429)


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [56]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.394, 'middle_aged': 0.357, 'senior': 0.457}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [57]:
#선정한 feature로 데이터프레임 split을 해주세요.

df_1 = pd_data[pd_data['age'] == 'middle_aged']
df_2 = pd_data[pd_data['age'] != 'middle_aged']

In [58]:
df_1.head() #split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [59]:
df_2.head() #split 결과 확인

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [60]:
# 각 변수를 대상으로 반복문 수행
# 결과는 소수점 이하 4자리까지 출력

gellot = ['middle_aged','youth_senior']

for num,i in enumerate([df_1,df_2]):
    
    if num == 1:
        print("#############################################################")
    
    label = i.columns[-1]
    features = list(i.columns[:-1])
    
    for feature in features:
        if get_attribute_gini_index(i,feature,label) == {}:
            continue
        print(f"Age: {gellot[num]} Minimum Gini index of {feature} : {min(get_attribute_gini_index(i,feature,label).items())}")

Age: middle_aged Minimum Gini index of income : ('high', 0.0)
Age: middle_aged Minimum Gini index of student : ('no', 0.0)
Age: middle_aged Minimum Gini index of credit_rating : ('fair', 0.0)
#############################################################
Age: youth_senior Minimum Gini index of age : ('youth', 0.48)
Age: youth_senior Minimum Gini index of income : ('high', 0.375)
Age: youth_senior Minimum Gini index of student : ('no', 0.32)
Age: youth_senior Minimum Gini index of credit_rating : ('fair', 0.417)


## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [68]:
from math import log2

def getEntropy(df, feature) :
    
    entropy = 0
    n = len(df[feature])
    
    for i in list(df[feature].value_counts()):
        entropy -= (i/n)*np.log2(i/n)
    
    return(entropy)

In [73]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [84]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}

    info_D = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    
    n = len(df[feature])
    
    for i in columns:
        preobs = 0
        for j in list(df[i].unique()):
            preobs += (sum(df[i] == j) /n)*getEntropy(df[df[i] == j],feature)
        
        result[i] = info_D - preobs
    
    return(result)

In [85]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}